# Example script for Hackathon

Within each cycle of active learning, you can:

1. Collect training data (original training data + your query data).

2. Train a prediction model to predict the DMS_score for each mutant (e.g., M0A).

3. Use the trained model to predict the score for all mutant in the test set.

4. Select query mutants for next round based on certain criteria. You may want to make sure you don't query the same mutant twice as you only have a limited chances of making queries in total.

In [167]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
import random
from copy import deepcopy
#!pip install pandas
import pandas as pd
from scipy.stats import spearmanr
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
## Added Packages
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
!nvidia-smi

Fri Mar 28 21:23:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:25:00.0 Off |                    0 |
| N/A   34C    P0             46W /  250W |       1MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 1. collect training data

Upload `sequence.fasta`, `train.csv`, and `test.csv` to the current runtime:

1. click the folder icon on the left

2. click the upload icon and upload the files to the current directory

In [80]:
with open('sequence.fasta', 'r') as f:
    data = f.readlines()

sequence_wt = data[1].strip()
sequence_wt

'MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLREKMRRRLESGDKWFSLEFFPPRTAEGAVNLISRFDRMAAGGPLYIDVTWHPAGDPGSDKETSSMMIASTAVNYCGLETILHMTCCRQRLEEITGHLHKAKQLGLKNIMALRGDPIGDQWEEEEGGFNYAVDLVKHIRSEFGDYFDICVAGYPKGHPEAGSFEADLKHLKEKVSAGADFIITQLFFEADTFFRFVKACTDMGITCPIVPGIFPIQGYHSLRQLVKLSKLEVPQEIKDVIEPIKDNDAAIRNYGIELAVSLCQELLASGLVPGLHFYTLNREMATTEVLKRLGMWTEDPRRPLPWALSAHPKRREEDVRPIFWASRPKSYIYRTQEWDEFPNGRWGNSSSPAFGELKDYYLFYLKSKSPKEELLKMWGEELTSEESVFEVFVLYLSGEPNRNGHKVTCLPWNDEPLAAETSLLKEELLRVNRQGILTINSQPNINGKPSSDPIVGWGPSGGYVFQKAYLEFFTSRETAEALLQVLKKYELRVNYHLVNVKGENITNAPELQPNAVTWGIFPGREIIQPTVVDPVSFMFWKDEAFALWIERWGKLYEEESPSRTIIQYIHDNYFLVNLVDNDFPLDNCLWQVVEDTLELLNRPTQNARETEAP'

In [81]:
len(sequence_wt)

656

In [82]:
def get_mutated_sequence(mut, sequence_wt):
    wt, pos, mt = mut[0], int(mut[1:-1]), mut[-1]
    sequence = deepcopy(sequence_wt)
    return sequence[:pos]+mt+sequence[pos+1:]

In [83]:
df_train = pd.read_csv('train.csv')
df_train['sequence'] = df_train.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
df_test

,mutant,sequence
0,V1D,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,V1Y,MYNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,V1C,MCNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,V1A,MANEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,V1E,MENEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
...,...,...
11319,P655S,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11320,P655T,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11321,P655V,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11322,P655A,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [84]:
df_test

,mutant,sequence
0,V1D,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,V1Y,MYNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,V1C,MCNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,V1A,MANEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,V1E,MENEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
...,...,...
11319,P655S,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11320,P655T,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11321,P655V,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11322,P655A,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [85]:
df_test = pd.read_csv('test.csv')
df_test['sequence'] = df_test.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))

In [140]:
# TODO: integrate the query data that you acquired each round into df_train
def filter_df(input_df, check):
    original_aa=[]
    altered_aa=[]
    position=[]
    score=[]
    changeTotal=[]
    for i, row in enumerate(input_df['mutant']):
        original_aa.append(list(input_df['mutant'][i])[0])
        altered_aa.append(list(input_df['mutant'][i])[-1])
        changeTotal.append(str(list(input_df['mutant'][i])[-1]) + str(list(input_df['mutant'][i])[0]))
        position.append(int(''.join(list(input_df['mutant'][i])[1:-1])))
        if check=="train":
            score.append(input_df['DMS_score'][i])
    if check=="train":
        df = pd.DataFrame({'Score':score,'OriginalAA': original_aa, 'AlteredAA': altered_aa, 'Position': position, 'changetotal':changeTotal})
    if check=="test":
        df = pd.DataFrame({'OriginalAA': original_aa, 'AlteredAA': altered_aa, 'Position': position, 'changetotal':changeTotal})

    return df


train_input=filter_df(df_train, "train")
test_input=filter_df(df_test, "test")

## Check positions

## Check AA Change
#set(train_input['changetotal']).difference(set(test_input['changetotal']))


#
dictOut=train_input['changetotal'].value_counts().to_dict()

newDF=pd.DataFrame([dictOut]).T.reset_index()
p1=newDF[newDF[0]==1]["index"]
p2=newDF[newDF[0]==2]["index"]

## Checking whether values in List
BasePairChange=set(list(p2) + list(p1))



,Score,OriginalAA,AlteredAA,Position,changetotal
0,0.2730,M,Y,0,YM
1,0.2857,M,W,0,WM
2,0.2153,M,V,0,VM
3,0.3122,M,T,0,TM
4,0.2180,M,S,0,SM
...,...,...,...,...,...
1078,0.5922,R,N,334,NR
1079,0.0412,R,H,334,HR
1080,0.4785,R,I,334,IR
1081,0.6796,R,K,334,KR


In [200]:
a=set(train_input['Position'])
b=set(test_input['Position'])
diffCheck=b.difference({0,
 48,
 87,
 88,
 89,
 112,
 126,
 141,
 148,
 151,
 152,
 157,
 158,
 182,
 185,
 189,
 191,
 193,
 195,
 196,
 216,
 220,
 221,
 224,
 227,
 228,
 242,
 247,
 250,
 251,
 252,
 253,
 254,
 255,
 258,
 276,
 289,
 290,
 292,
 295,
 296,
 297,
 298,
 302,
 304,
 305,
 308,
 309,
 311,
 314,
 315,
 316,
 318,
 319,
 320,
 331,
 334,
 335,
 336,
 347})
outConvertPos=[str(i) for i in diffCheck]

In [251]:
#out=test_input[test_input['Position'].isin(diffCheck)]
#test_input['Position'].isin(diffCheck)
outFilter=test_input[test_input['changetotal'].isin(BasePairChange)]
import random

#random.sample(outFilter.index], 100)
#print(random.sample([1,2,3,4,5], 5))
outDF=random.sample(list(outFilter.index), 100)

#outDF
df = pd.DataFrame(outFilter, index=outDF)
pd.DataFrame(df).to_csv("test_query.csv")

In [234]:
TestUnique=list(set(test_input['Position']))
TrainUnique=list(set(train_input['Position']))
result = [x for x in TestUnique if x not in TrainUnique]
print(result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 149, 150, 153, 154, 155, 156, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 186, 187, 188, 190, 192, 194, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 217, 218, 219, 222, 223, 225, 226, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 243, 244, 245, 246, 248

In [178]:
A = {1, 2, 3, 4, 5}
B = {3, 4, 5, 6, 7}
C = {5, 6, 7, 8, 9}

res = A.difference(B, C)
a

{0,
 48,
 87,
 88,
 89,
 112,
 126,
 141,
 148,
 151,
 152,
 157,
 158,
 182,
 185,
 189,
 191,
 193,
 195,
 196,
 216,
 220,
 221,
 224,
 227,
 228,
 242,
 247,
 250,
 251,
 252,
 253,
 254,
 255,
 258,
 276,
 289,
 290,
 292,
 295,
 296,
 297,
 298,
 302,
 304,
 305,
 308,
 309,
 311,
 314,
 315,
 316,
 318,
 319,
 320,
 331,
 334,
 335,
 336,
 347}

## 2. Train a prediction model

Here, we provided a linear regression model and used one-hot encoding to encode each variant. You would need to build your own model to achieve better performances.

Hint: you can perform cross-validation on the training set to evaluate your predictor before making predictions on the test set.

In [52]:
'''hyperparameters'''

seq_length = 656
seed = 0 # seed for splitting the validation set
val_ratio = 0.2 # proportion of validation set

In [16]:
class ProteinDataset(Dataset):
    def __init__(self, df, istrain=True):

        alphabet = 'ACDEFGHIKLMNPQRSTVWY'
        map_a2i = {j:i for i,j in enumerate(alphabet)}
        map_i2a = {i:j for i,j in enumerate(alphabet)}

        self.df = df

        self.num_samples = len(self.df)
        self.seq_length = len(self.df.sequence.values[0])
        self.num_channels = 20

        # TODO: replace one-hot encodings with your own encodings
        self.encodings = np.zeros((self.num_samples, self.num_channels, self.seq_length)).astype(np.float32)
        self.targets = np.zeros(self.num_samples).astype(np.float32)

        if istrain:
            for it, (seq,target) in enumerate(self.df[['sequence', 'DMS_score']].values):
                for i,aa in enumerate(seq):
                    self.encodings[it,map_a2i[aa], i  + (20*i)] = 1
                self.targets[it] = target

            self.encodings = self.encodings.astype(np.float32)
            self.targets = self.targets.astype(np.float32)
        else:
            for it, seq in enumerate(self.df['sequence'].values):
                for i,aa in enumerate(seq):
                    self.encodings[it,map_a2i[aa],i  + (20*i)] = 1
            self.encodings = self.encodings.astype(np.float32)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return torch.tensor(self.encodings[idx]), torch.tensor(self.targets[idx])

In [17]:
train_dataset = ProteinDataset(df_train)
test_dataset = ProteinDataset(df_test, istrain=False)

# split validation set
train_dataset, val_dataset = train_test_split(train_dataset, test_size=val_ratio, random_state=seed, shuffle=True)

# TODO: revise according to your own model

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)


IndexError: index 672 is out of bounds for axis 2 with size 656

In [ ]:
class initial_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(initial_model, self).__init__()
        self.fc1=nn.Linear(input_dim, hidden_dim)
        self.fc2=nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu=nn.ReLU()

    def forward(self, x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x) 
        return x


def evaluate(model, dataloader, criterion, device='cuda:0'):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for sequence, fitness in dataloader:
            input_values = sequence.to(device)
            label_values = fitness.to(device).view(-1)

            outputs = model(input_values)
            loss = criterion(outputs, label_values)
            total_loss += loss.item()

    return total_loss / len(dataloader)
    
def train(model, dataloader_train, dataloader_val, epochs=200, lr=0.002, patience=50, device='cuda:0'):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    patience_counter = 0
    best_ckpt = model.state_dict()

    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for sequence, fitness in dataloader_train:
            input_values = sequence.to(device)
            label_values = fitness.to(device).view(-1)

            optimizer.zero_grad()
            outputs = model(input_values)
            loss = criterion(outputs, label_values)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_loss = evaluate(model, dataloader_val, criterion, device)
        print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Val Loss={val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_ckpt = model.state_dict()
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    return best_ckpt

# Initialize model
input_dim = 656
hidden_dim = 32
output_dim = 1

model = initial_model(input_dim, hidden_dim, output_dim)

# Train the model
best_ckpt = train(model, dataloader_train=train_loader, dataloader_val=val_loader, epochs=100, lr=1e-2, patience=250)


In [ ]:
# TODO: build your own prediction model to replace linear regression
# Hint: don't forget to use the validation set: you can either integrate the validation data into the training set or use it separately for early stopping

X_train, y_train = next(iter(train_loader))

regressor = LinearRegression()
#regressor=RandomForestRegressor()

#nn.Sequential()

regressor.fit(X_train.view(X_train.size(0), -1).numpy(), y_train.numpy())

X_test, _ = next(iter(test_loader))
y_test_pred = regressor.predict(X_test.view(X_test.size(0), -1).numpy())


In [ ]:
df_test['DMS_score_predicted'] = y_test_pred
df_test

In [ ]:
df_test[['mutant', 'DMS_score_predicted']].to_csv('test_predictions.csv')

## 3. Select query for next round

In [ ]:
df_test.sort_values('DMS_score_predicted', ascending=False).head(100)

In [ ]:
# Example: randomly select 100 test variants to be queried.
# Note: random selection may not be a good strategy
# TODO: select query mutants for the next round based on your own criteria

querys = np.random.choice(df_test.mutant.values, size=100, replace=False)
querys


In [ ]:
with open('query.txt', 'w') as f:
    for mutant in querys:
        f.write(mutant+'\n')